# Merge data #
For the SentirEs project this code is gathering the ratings six sensory modalities from the partipants:
Audition, Gustatory, Haptic, Interoception, Olfaction, and Vision. The main purpose of this code is to read a folder of csv files
(all the partipants resonses) and concatonate it toda to form one long csv file we can use in our data analysis section. 


In [12]:
import pandas as pd
import glob
import os

In [10]:
def combine_csv_from_folder(folder_path, columns, output_file):
    
  file_paths = glob.glob(f"{folder_path}/*.csv")
  dataframes = []

  for file_path in file_paths:
      df = pd.read_csv(file_path, usecols=columns)
      dataframes.append(df)

  combined_df = pd.concat(dataframes, ignore_index=True)
  combined_df.to_csv(output_file, index=True)

# Example usage:
folder_path = "/Users/soniasimon/Documents/thesis/selecting_files/data"
columns = ["Word", "Audition.response", "Gustatory.response", "Haptic.response", "Interoception.response", "Olfaction.response", "Vision.response"]
output_file = "/Users/soniasimon/Documents/thesis/selecting_files/output"
combine_csv_from_folder(folder_path, columns, output_file)

In [19]:
def add_column_to_csv(source_csv, target_csv, new_column_name):
  """Adds a column from source_csv to target_csv.

  Args:
    source_csv: Path to the CSV file containing the new column.
    target_csv: Path to the target CSV file where the new column will be added.
    new_column_name: Name of the column to be added.
  """

  # Read both CSV files into DataFrames
  source_df = pd.read_csv(source_csv)
  target_df = pd.read_csv(target_csv)

  # Ensure both DataFrames have the same number of rows
  if len(source_df) != len(target_df):
    raise ValueError("DataFrames must have the same number of rows.")

  # Add the new column to the target DataFrame
  target_df[new_column_name] = source_df[new_column_name]

  # Save the updated DataFrame to a new CSV file
  target_df.to_csv(target_csv, index=False)

In [23]:

def process_csv_files_in_folder(folder_path_2, target_csv, new_column_name):
 
    # Get a list of CSV files in the folder
  csv_files = [f for f in os.listdir(folder_path_2) if f.endswith('.csv')]

  # Process each CSV file
  for csv_file in csv_files:
    source_csv = os.path.join(folder_path, csv_file)
    add_column_to_csv(source_csv, target_csv, new_column_name)

# Example usage
folder_path_2 = "/Users/soniasimon/Documents/thesis/selecting_files/"
target_csv = "word_category.cvs"
new_column_name = "Word Category"

process_csv_files_in_folder(folder_path, target_csv, new_column_name)

FileNotFoundError: [Errno 2] No such file or directory: 'word_category.cvs'